<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 1 - Renata

## Prerequisites

Make sure the prerequisites in [CL_LMDA_prerequisites](https://github.com/laelgelc/laelgelc/blob/main/CL_LMDA_prerequisites.ipynb) are satisfied.

## Dataset

Please download the following dataset (Right-click on the link and choose `Save link as` to download the corresponding file):
- [cl_st1_renata_truthbrush_20240616_raw_data_prep.jsonl](https://laelgelcrenata.s3.sa-east-1.amazonaws.com/cl_st1_renata_truthbrush_20240616_raw_data_prep.jsonl)

## Importing the required libraries

In [1]:
import pandas as pd
import demoji
import re
import os
from collections import Counter

## Data wrangling

### Importing the tweet raw data into a dataframe

In [2]:
df_tweets_raw_data = pd.read_json('cl_st1_renata_truthbrush_20240616_raw_data_prep.jsonl', lines=True)

In [3]:
df_tweets_raw_data.head(5)

,created_at,author_id,username,tweet_url,content,text
0,2024-06-16 21:27:19.392,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262841581...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Sunday Live: Hollywood Comes to...
1,2024-06-16 18:00:57.062,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262760432...,<p>Watch: Alex Jones Calls Out Globalist Tyran...,Watch: Alex Jones Calls Out Globalist Tyrants ...
2,2024-06-15 15:02:15.900,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262123939...,<p>Alex Jones Emergency Saturday Broadcast: Le...,Alex Jones Emergency Saturday Broadcast: Learn...
3,2024-06-15 00:08:13.055,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261772385...,<p>Exclusive: Alex Jones Makes First Statement...,Exclusive: Alex Jones Makes First Statements A...
4,2024-06-14 17:19:17.792,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261611591...,"<p><a href=""https://truthsocial.com/tags/AlexJ...","#AlexJonesShow LIVE: Tucker Carlson, Russell B..."


In [4]:
df_tweets_raw_data.shape

(762993, 6)

#### Inspecting a few tweets

In [5]:
inspected_row = 14
print('username:' + df_tweets_raw_data.loc[inspected_row, 'username'])
print('text:' + df_tweets_raw_data.loc[inspected_row, 'text'])
print('tweet_url:' + df_tweets_raw_data.loc[inspected_row, 'tweet_url'])

username:AlexJones
text:RT @infowarsGET READY: Tucker Carlson LIVE Nationwide Tour — Coming This Fall! LEARN MORE  https://www.infowars.com/posts/get-ready-tucker-carlson-live-nationwide-tour-coming-this-fall/
tweet_url:https://truthsocial.com/@AlexJones/112604587244214528


### Inspecting the dataset and eliminating malformed data

#### Checking if data types are consistent

In [6]:
df_tweets_raw_data.dtypes

created_at    datetime64[ns]
author_id              int64
username              object
tweet_url             object
content               object
text                  object
dtype: object

#### Identifying rows that are empty in column `text`

In [7]:
print(df_tweets_raw_data['text'].isnull().sum())

0


In [8]:
df_tweets_raw_data[df_tweets_raw_data['text'].isnull()]

,created_at,author_id,username,tweet_url,content,text


#### Dropping the rows that are empty in the column `text`

In [9]:
# Drop the rows whose column 'text' is NaN
df_tweets_raw_data = df_tweets_raw_data.dropna(subset=['text'])

# Reset the index
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

In [10]:
print(df_tweets_raw_data['text'].isnull().sum())

0


#### Removing specific Unicode characters

The dataset may need to be cleaned of invisible Unicode characters.

##### Detecting `U+2066` and `U+2069` characters

- [U+2066](https://www.compart.com/en/unicode/U+2066)
- [U+2069](https://www.compart.com/en/unicode/U+2069)

Please refer to:
- [Python RegEx](https://www.w3schools.com/python/python_regex.asp)
- [regex101](https://regex101.com/)
- [RegExr](https://regexr.com/)

In [11]:
# Defining a function to detect specific Unicode characters
def extract_unicode_characters(df, column_name):
    unicode_chars = Counter()  # Initialize a Counter to store Unicode character counts

    for value in df[column_name]:
        if isinstance(value, str):
            # Use RegEx to find non-ASCII characters (Unicode)
#            non_ascii_chars = re.findall(r'[^\x00-\x7F]+', value)
            # Use RegEx to find specific Unicode characters - adjust the expression accordingly
            specific_unicode_chars = re.findall(r'[\u2066\u2069]', value)
            unicode_chars.update(specific_unicode_chars)

    return unicode_chars

# Inspect the dataframe for specific Unicode characters
unicode_counts = extract_unicode_characters(df_tweets_raw_data, 'text')

# Print the results
for char, count in unicode_counts.items():
    print(f'Character {char}: Count = {count}')

Character ⁦: Count = 93
Character ⁩: Count = 70


##### Removing `U+2066` and `U+2069` characters

In [12]:
# Defining a function to remove specific Unicode characters
def remove_specific_unicode(input_line):
    # Using RegEx to replace specific Unicode characters - adjust the expression accordingly
    cleaned_line = re.sub(r'[\u2066\u2069]', '', input_line)
    return cleaned_line

# Removing specific Unicode characters
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_specific_unicode)

#### Replacing the `LF` character by a space

Some tweets, especially the retweeted ones, contain multiple lines of text.

In [13]:
# Defining a function to replace the `LF` character by a space
def remove_cr_lf(input_line):
    # Using RegEx to replace LF by a space
    cleaned_line = re.sub(r'\n', ' ', input_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_cr_lf)

In [14]:
df_tweets_raw_data

,created_at,author_id,username,tweet_url,content,text
0,2024-06-16 21:27:19.392,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262841581...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Sunday Live: Hollywood Comes to...
1,2024-06-16 18:00:57.062,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262760432...,<p>Watch: Alex Jones Calls Out Globalist Tyran...,Watch: Alex Jones Calls Out Globalist Tyrants ...
2,2024-06-15 15:02:15.900,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262123939...,<p>Alex Jones Emergency Saturday Broadcast: Le...,Alex Jones Emergency Saturday Broadcast: Learn...
3,2024-06-15 00:08:13.055,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261772385...,<p>Exclusive: Alex Jones Makes First Statement...,Exclusive: Alex Jones Makes First Statements A...
4,2024-06-14 17:19:17.792,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261611591...,"<p><a href=""https://truthsocial.com/tags/AlexJ...","#AlexJonesShow LIVE: Tucker Carlson, Russell B..."
...,...,...,...,...,...,...
762988,2022-02-18 22:22:42.617,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/107821332...,<p>Remember this is SOCIAL media. It’s meant ...,Remember this is SOCIAL media. It’s meant to ...
762989,2022-02-18 17:16:21.149,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/107820127...,<p>The dream has become a reality. You have yo...,The dream has become a reality. You have your ...
762990,2022-04-12 02:04:45.040,107838824115445312,warrendavidson,https://truthsocial.com/@warrendavidson/108116...,"<p>NATO should be reinforced, not re-imagined ...","NATO should be reinforced, not re-imagined wit..."
762991,2022-04-06 17:02:49.117,107838824115445312,warrendavidson,https://truthsocial.com/@warrendavidson/108086...,"<p>Yesterday, I was one of 63 members of Congr...","Yesterday, I was one of 63 members of Congress..."


### Dropping duplicates

#### Retweets

Retweets bear the RegEx pattern `'\bRT @\w+'gm` at the beginning of the column `text`.

In [15]:
# Creating a boolean mask for filtering - it is preceded by '~' to invert the selection
mask = ~df_tweets_raw_data['text'].str.contains(r'\bRT @\w+', regex=True)

# Applying the mask to overwrite the raw data dataframe with non retweeted tweets
df_tweets_raw_data = df_tweets_raw_data[mask]
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(593948, 6)

#### Duplicate tweets

The dataset was build in a way that if a certain tweet had more than one photo, one copy of the tweet was included per unique photo. Since we are concerned with analysing just the text, those duplicates should be removed. Tweets that bear the same 'tweet_url' are duplicates - we are going to keep only the first.

In [16]:
df_tweets_raw_data.drop_duplicates(subset='tweet_url', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(593948, 6)

#### @mentioned tweets

A few users @mention copies of tweets towards other specific users creating multiple copies of the same tweet - those duplicates should be removed.

In [17]:
# Create a new column 'no_mention' containing the contents of the column 'text' without any preceding @mentions
df_tweets_raw_data['no_mention'] = df_tweets_raw_data['text'].str.replace(r'@\w+\s*', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_raw_data.drop_duplicates(subset='no_mention', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(504181, 7)

## Sampling the raw data according to filtering expressions

**Important**: This step requires close attention. Review the `filter_words` list by adding/removing words (make sure to have them lower-case) that potentially attract the discourses you want to study and aim to keep the size of the target corpus between 20 and 50 thousand tweets. The number of tweets is given by the number of rows in `<x> rows × <y> columns` at the bottom of the `df_tweets_filtered` dataframe display. Reiterate the execution of the following lines until the target corpus is sized accordingly. Exceeding the limit may cause the computer to run out of RAM memory when running the function `datamatrix.sh` ahead.

Note: The words proposed in the `filter_words` list below are temporary, as agreed, and resulted in a target corpus above the limit. As a result, it was required a virtual computer with 32 GB of RAM memory hired from Amazon Web Services (instance type: m5a.2xlarge).

In [18]:
# Defining the filtering expressions
#filter_words = ['arma', 'pátria', 'ladrão', 'cristão', 'comunista', 'família', 'liberdade', 'conservador', 'deus']
filter_words = ['chinese', 'communist', 'democrat', 'maga', 'president', 'vaccine', 'vax']

# Creating a boolean mask for filtering
mask = df_tweets_raw_data['text'].str.contains('|'.join(filter_words), case=False)

# Applying the mask to create 'df_tweets_filtered'
df_tweets_filtered = df_tweets_raw_data[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)

In [19]:
df_tweets_filtered

,created_at,author_id,username,tweet_url,content,text,no_mention
0,2024-06-13 23:33:42.367,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261192584...,<p>BREAKING! Democrats File To Take Alex Jones...,BREAKING! Democrats File To Take Alex Jones’ X...,BREAKING! Democrats File To Take Alex Jones’ X...
1,2024-06-12 16:21:08.882,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11260456264...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow LIVE: Ships & Submarine Loaded ...,#AlexJonesShow LIVE: Ships & Submarine Loaded ...
2,2024-06-11 23:37:11.374,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11260061491...,<p>BREAKING: Hunter Biden Found Guilty As Demo...,BREAKING: Hunter Biden Found Guilty As Democra...,BREAKING: Hunter Biden Found Guilty As Democra...
3,2024-06-11 23:35:01.444,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11260060640...,<p>Avian Influenza Added to Flu Vaccine in 200...,Avian Influenza Added to Flu Vaccine in 2009 ...,Avian Influenza Added to Flu Vaccine in 2009 ...
4,2024-06-10 23:37:00.009,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11259495186...,<p>Breaking: Federal Court Rules Covid Shots A...,Breaking: Federal Court Rules Covid Shots Are ...,Breaking: Federal Court Rules Covid Shots Are ...
...,...,...,...,...,...,...,...
80611,2024-03-22 16:09:15.760,108379772378225680,therealkidrock,https://truthsocial.com/@therealkidrock/112140...,"<p>Psalm 34:21<br/>Evil shall slay the wicked,...","Psalm 34:21Evil shall slay the wicked,And thos...","Psalm 34:21Evil shall slay the wicked,And thos..."
80612,2024-02-20 22:46:10.305,108379772378225680,therealkidrock,https://truthsocial.com/@therealkidrock/111966...,"<p><span class=""quote-inline""><br/>RT: https:/...",RT: https://truthsocial.com/users/realDonaldTr...,RT: https://truthsocial.com/users/realDonaldTr...
80613,2022-06-06 16:53:53.448,108379772378225680,therealkidrock,https://truthsocial.com/@therealkidrock/108431...,<p>Amen to that Mr. President!!!! <span cla...,Amen to that Mr. President!!!! @realDonaldT...,Amen to that Mr. President!!!!
80614,2023-10-16 22:13:00.485,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/111246991...,"<p><span class=""quote-inline""><br/>RT: https:/...",RT: https://truthsocial.com/users/BidenHQ/stat...,RT: https://truthsocial.com/users/BidenHQ/stat...


### Exporting the filtered data into a file for inspection

In [20]:
df_tweets_filtered.to_csv('tweets_emojified.tsv', sep='\t', index=False)

## Replacing emojis

### Demojifying the column `text`

In [21]:
# Defining a function to demojify a string
def demojify_line(input_line):
    demojified_line = demoji.replace_with_desc(input_line, sep='<em>')
    return demojified_line

df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(demojify_line)

#### Exporting the filtered data into a file for inspection

In [22]:
df_tweets_filtered.to_csv('tweets_demojified1.tsv', sep='\t', index=False)

### Separating the demojified strings with spaces

In [23]:
# Defining a function to separate the demojified strings with spaces
def preprocess_line(input_line):
    # Add a space before the first delimiter '<em>', if it is not already preceded by one
    preprocessed_line = re.sub(r'(?<! )<em>', ' <em>', input_line)
    # Add a space after the first delimiter '<em>', if it is not already followed by one
    preprocessed_line = re.sub(r'<em>(?! )', '<em> ', preprocessed_line)
    return preprocessed_line

# Separating the demojified strings with spaces
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(preprocess_line)

#### Exporting the filtered data into a file for inspection

In [24]:
df_tweets_filtered.to_csv('tweets_demojified2.tsv', sep='\t', index=False)

### Formatting the demojified strings

In [25]:
# Defining a function to format the demojified string
def format_demojified_string(input_line):
    # Defining a function to format the demojified string using RegEx
    def process_demojified_string(s):
            # Lowercase the string
            s = s.lower()
            # Replace spaces and colons followed by a space with underscores
            s = re.sub(r'(: )| ', '_', s)
            # Add the appropriate prefixes and suffixes
            s = f'EMOJI{s}e'
            return s

    # Use RegEx to find and process each demojified string
    processed_line = re.sub(r'<em>(.*?)<em>', lambda match: process_demojified_string(match.group(1)), input_line)
    return processed_line

# Formatting the demojified strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(format_demojified_string)

### Replacing the `pipe` character by the `-` character in the `text` column

Further on, a few columns of the dataframe are going to be exported into the file `tweets.txt` whose columns need to be delimited by the `pipe` character. Therefore, it is recommended that any occurrences of the `pipe` character in the `text` column are replaced by another character.

In [26]:
# Defining a function to replace the 'pipe' character by the '-' character
def replace_pipe_with_hyphen(input_string):
    modified_string = re.sub(r'\|', '-', input_string)
    return modified_string

# Replacing the 'pipe' character by the '-' character
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(replace_pipe_with_hyphen)


#### Exporting the filtered data into a file for inspection

In [27]:
df_tweets_filtered.to_csv('tweets_demojified3.tsv', sep='\t', index=False)

## Tokenising

Please refer to [What is tokenization in NLP?](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/).

In [28]:
# Defining a function to tokenise a string
def tokenise_string(input_line):
    # Replace URLs with placeholders
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b'
    placeholder = '<URL>'  # Choose a unique placeholder
    urls = re.findall(url_pattern, input_line)
    tokenised_line = re.sub(url_pattern, placeholder, input_line)  # Replace URLs with placeholders
    
    # Replace curly quotes with straight ones
    tokenised_line = tokenised_line.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    # Separate common punctuation marks with spaces
    tokenised_line = re.sub(r'([.\!?,"\'/()])', r' \1 ', tokenised_line)
    # Add a space before '#'
    tokenised_line = re.sub(r'(?<!\s)#', r' #', tokenised_line)  # Add a space before '#' if it is not already preceded by one
    # Reduce extra spaces by a single space
    tokenised_line = re.sub(r'\s+', ' ', tokenised_line)
    
    # Replace the placeholders with the respective URLs
    for url in urls:
        tokenised_line = tokenised_line.replace(placeholder, url, 1)
    
    return tokenised_line

# Tokenising the strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(tokenise_string)

## Creating the files `file_index.txt` and `tweets.txt`

### Creating column `text_id`

In [29]:
df_tweets_filtered['text_id'] = 't' + df_tweets_filtered.index.astype(str).str.zfill(6)

### Creating column `conversation`

In [30]:
df_tweets_filtered['author_id'] = df_tweets_filtered['author_id'].astype(str)

In [31]:
df_tweets_filtered['conversation'] = 'v:' + df_tweets_filtered['author_id'].str.replace('id_', '')

### Creating column `date`

In [32]:
# Convert 'created_at' to datetime format
df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])

# Extract the date part (without time) into a new column 'date'
df_tweets_filtered['date'] = df_tweets_filtered['created_at'].dt.date

# Add the prefix 'd:' to the 'date' values
df_tweets_filtered['date'] = 'd:' + df_tweets_filtered['date'].astype(str)

### Creating column `text_url`

In [33]:
df_tweets_filtered['text_url'] = 'url:' + df_tweets_filtered['tweet_url']

### Creating column `user`

In [34]:
df_tweets_filtered['user'] = 'u:' + df_tweets_filtered['username']

### Creating column `content`

In [35]:
df_tweets_filtered['content'] = 'c:' + df_tweets_filtered['text']

### Reordering the created columns

Please refer to:
- [Python - List Comprehension 1](https://www.w3schools.com/python/python_lists_comprehension.asp)
- [Python - List Comprehension 2](https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)

In [36]:
# Reorder the columns (we use list comprehension to create a list of all columns except 'text_id', 'variable', 'date' and 'text_url')
df_tweets_filtered = df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url', 'user', 'content'] + [col for col in df_tweets_filtered.columns if col not in ['text_id', 'conversation', 'date', 'text_url', 'user', 'content']]]

In [37]:
df_tweets_filtered

,text_id,conversation,date,text_url,user,content,created_at,author_id,username,tweet_url,text,no_mention
0,t000000,v:107838014712814240,d:2024-06-13,url:https://truthsocial.com/@AlexJones/1126119...,u:AlexJones,c:BREAKING ! Democrats File To Take Alex Jones...,2024-06-13 23:33:42.367,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261192584...,BREAKING ! Democrats File To Take Alex Jones '...,BREAKING! Democrats File To Take Alex Jones’ X...
1,t000001,v:107838014712814240,d:2024-06-12,url:https://truthsocial.com/@AlexJones/1126045...,u:AlexJones,c: #AlexJonesShow LIVE: Ships & Submarine Load...,2024-06-12 16:21:08.882,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11260456264...,#AlexJonesShow LIVE: Ships & Submarine Loaded...,#AlexJonesShow LIVE: Ships & Submarine Loaded ...
2,t000002,v:107838014712814240,d:2024-06-11,url:https://truthsocial.com/@AlexJones/1126006...,u:AlexJones,c:BREAKING: Hunter Biden Found Guilty As Democ...,2024-06-11 23:37:11.374,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11260061491...,BREAKING: Hunter Biden Found Guilty As Democra...,BREAKING: Hunter Biden Found Guilty As Democra...
3,t000003,v:107838014712814240,d:2024-06-11,url:https://truthsocial.com/@AlexJones/1126006...,u:AlexJones,c:Avian Influenza Added to Flu Vaccine in 2009...,2024-06-11 23:35:01.444,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11260060640...,Avian Influenza Added to Flu Vaccine in 2009 h...,Avian Influenza Added to Flu Vaccine in 2009 ...
4,t000004,v:107838014712814240,d:2024-06-10,url:https://truthsocial.com/@AlexJones/1125949...,u:AlexJones,c:Breaking: Federal Court Rules Covid Shots Ar...,2024-06-10 23:37:00.009,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11259495186...,Breaking: Federal Court Rules Covid Shots Are ...,Breaking: Federal Court Rules Covid Shots Are ...
...,...,...,...,...,...,...,...,...,...,...,...,...
80611,t080611,v:108379772378225680,d:2024-03-22,url:https://truthsocial.com/@therealkidrock/11...,u:therealkidrock,"c:Psalm 34:21Evil shall slay the wicked , And ...",2024-03-22 16:09:15.760,108379772378225680,therealkidrock,https://truthsocial.com/@therealkidrock/112140...,"Psalm 34:21Evil shall slay the wicked , And th...","Psalm 34:21Evil shall slay the wicked,And thos..."
80612,t080612,v:108379772378225680,d:2024-02-20,url:https://truthsocial.com/@therealkidrock/11...,u:therealkidrock,c:RT: https://truthsocial.com/users/realDonald...,2024-02-20 22:46:10.305,108379772378225680,therealkidrock,https://truthsocial.com/@therealkidrock/111966...,RT: https://truthsocial.com/users/realDonaldTr...,RT: https://truthsocial.com/users/realDonaldTr...
80613,t080613,v:108379772378225680,d:2022-06-06,url:https://truthsocial.com/@therealkidrock/10...,u:therealkidrock,c:Amen to that Mr . President ! ! ! ! @realDon...,2022-06-06 16:53:53.448,108379772378225680,therealkidrock,https://truthsocial.com/@therealkidrock/108431...,Amen to that Mr . President ! ! ! ! @realDonal...,Amen to that Mr. President!!!!
80614,t080614,v:107759501782461328,d:2023-10-16,url:https://truthsocial.com/@truthsocial/11124...,u:truthsocial,c:RT: https://truthsocial.com/users/BidenHQ/st...,2023-10-16 22:13:00.485,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/111246991...,RT: https://truthsocial.com/users/BidenHQ/stat...,RT: https://truthsocial.com/users/BidenHQ/stat...


### Creating the file `file_index.txt`

In [38]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url']].to_csv('file_index.txt', sep=' ', index=False, header=False, encoding='utf-8', lineterminator='\n')

### Creating the file `tweets.txt`

In [39]:
folder = 'tweets'
try:
    os.mkdir(folder)
    print(f'Folder {folder} created!')
except FileExistsError:
    print(f'Folder {folder} already exists')

Folder tweets created!


Note: The parameters `doublequote=False` and `escapechar=' '` are required to avoid that the column content is doublequoted with '"' in sentences that use characters that need to be escaped such as double quote '"' itself - this causes a malformed response from TreeTagger.

In [40]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'user', 'content']].to_csv(f'{folder}/tweets.txt', sep='|', index=False, header=False, encoding='utf-8', lineterminator='\n', doublequote=False, escapechar=' ')

## Tagging with TreeTagger

- On Visual Studio Code (VS Code), open the folder where your project is located with `Open Folder...`
- Open a WSL Ubuntu Terminal on VS Code
- **Important**: Activate the `my_env` Python environment by executing `source "$HOME"/my_env/bin/activate`
- Proceed as indicated

Note: You have to download and open this Jupyter Notebook on JupyterLab (provided as part of Anaconda Distribution) to visualise the procedure

Purpose: Annotate the texts in `tweets/tweets.txt` with part-of-speech and lemma information.
- Input
    - `file_index.txt`
    - `tweets/tweets.txt`
- Output
    - `tweets/tagged.txt`

## Processing `tokenstypes`

Purpose: Capture the content tokens (specific occurrences of words) and the content types (general concept of words) from `tweets/tagged.txt`.
- Input
    - `file_index.txt`
    - `tweets/tagged.txt`
- Output
    - `tweets/tokens.txt`
    - `tweets/types.txt`

## Processing `toplemmas`

Purpose: Determine the 1.000 top lemmas. **Important**: This process requires manual inspection. Non-meaningful lemmas should be excluded by updating `stoplist.sed` and reiterating the processing. Proceed as indicated in this [video tutorial](https://youtu.be/4UIPpdoozP0?si=_3md9w79njZY86PE), from 20:05 to 25:16.
- Input
    - `tweets/types.txt`
    - `stoplist.sed`: List of rules that allows the exclusion of a certain lemmas
- Output
    - `selectedwords` = `var_index.txt`

## Processing `sas`

Purpose: Prepare input data for processing in SAS.
- Input
    - `tweets/types.txt`
    - `selectedwords`
    - `file_index.txt`
- Output
    - `columns`
    - `sas/data.txt`
    - `sas/dates.txt`
    - `sas/wcount.txt`

## Processing `datamatrix`

Purpose: Prepares input data for calculating the correlation matrix.
- Input
    - `file_index.txt`
    - `columns`
    - `selectedwords`
- Output
    - `file_ids.txt`
    - `data.csv`

## Processing `correlationmatrix`

Purpose: Calculates the correlation matrix.
- Input
    - `data.csv`
- Output
    - `correlation`

## Processing `formats`

Purpose: Prepare input data for processing in SAS.
- Input
    - `data.csv`
    - `selectedwords`
- Output
    - `sas/corr.txt`
    - `sas/word_labels_format.sas`

## Processing the statistical procedures on SAS

- Log in to your [SAS OnDemand for Academics](https://welcome.oda.sas.com/) account
- Proceed as indicated in this [video tutorial](https://youtu.be/I3u9zD3jyOA?si=68uIKVc2iusGG2KY)

## Processing `examples`

Purpose: Extract examples for analysis.
- Input
    - `sas/output_"$project"/loadtable.html`
    - `sas/output_"$project"/"$project"_scores.tsv`
    - `sas/output_"$project"/"$project"_scores_only.tsv`
- Output
    - `examples/factors`
    - `example files`

## Results

Right-click on the link and choose `Save link as` to download the corresponding file.

- [CL_St1_Renata_Results.tar.gz](https://laelgelcrenata.s3.sa-east-1.amazonaws.com/CL_St1_Renata_Results.tar.gz)

Note:
- Proceed as follows to extract the files from the archive `CL_St1_Renata_Results.tar.gz`
- You have to download and open this Jupyter Notebook on JupyterLab (provided as part of Anaconda Distribution) to visualise the procedure